In [59]:
# from yf_utils import _2_split_train_val_test, _3_random_slices, _4_lookback_slices
from yf_utils import _5_perf_ranks, _6_grp_tuples_sort_sum
from myUtils import pickle_load

# verbose = False  # True prints more output
verbose = True  # True prints more output

path_dir = "C:/Users/ping/MyDrive/stocks/yfinance/"
path_data_dump = path_dir + "VSCode_dump/"
fp_df_close_clean = 'df_close_clean'
df_close_clean = pickle_load(path_data_dump, fp_df_close_clean)

In [60]:
# number of days to lookback from last df date for performance evaluation
# days_lookbacks = [30, 120, 60, 240]
# days_lookbacks = [24, 48, 95, 190]
days_lookbacks = [16, 32, 64, 128]  # corresponds to 0.8, 1.5, 3.0, 6.1 months
# days_lookbacks = [15, 30, 60, 120]  # corresponds to 0.8, 1.5, 3.0, 6.1 months
# days_lookbacks = [30, 60, 120]  # corresponds to 0.8, 1.5, 3.0, 6.1 months
days_lookbacks.sort()  # sort list of integers in ascending order

# number of the most-common symbols from days_lookbacks' performance rankings to keep
n_top_syms = 30  

syms_start = 0  # slice start of set_most_common_syms
syms_end = 5  #  slice end of set_most_common_syms

In [61]:
# grp_top_set_syms_n_freq = []  # list of lists of top_set_symbols_n_freq, there are n_samples lists in list
# grp_top_set_syms = []  # list of lists of top_set_symbols, there are n_samples lists in list
grp_most_common_syms = []  # accumulates the best performing symbols from each days_lookback
grp_perf_ranks = []  # accumulates the best performance ranking symbols from each days_lookback 
grp_dates = []  # dates of days_lookbacks 

for d_lookback in days_lookbacks:
  df = df_close_clean.iloc[-d_lookback:]
  perf_ranks, most_common_syms = _5_perf_ranks(df, n_top_syms=n_top_syms)  # get the best performance symbols and rankings
  grp_most_common_syms.append(most_common_syms)
  grp_perf_ranks.append(perf_ranks)  
  grp_dates.append(df.index[0].strftime("%Y-%m-%d")) 

grp_dates.append(df.index[-1].strftime("%Y-%m-%d"))  # get df's last date
grp_dates.sort()

In [62]:
# flatten list of lists of (symbol:frequency)
flat_grp_most_common_syms = [val for sublist in grp_most_common_syms for val in sublist]
# group symbols from set of days_lookbacks (i.e. lb_slices) and sum frequency of the symbols
set_most_common_syms = _6_grp_tuples_sort_sum(flat_grp_most_common_syms, reverse=True)
# get the top few most-frequent symbol:frequency pairs
top_set_syms_n_freq = set_most_common_syms[0:n_top_syms]
# get symbols from top_set_syms_n_freq
top_set_syms = [i[0] for i in top_set_syms_n_freq[syms_start:syms_end]]

In [63]:
print(f'grp_dates: {grp_dates}')
print(f'top_set_syms_n_freq: {top_set_syms_n_freq}')
print(f'top_set_syms: {top_set_syms}')

grp_dates: ['2022-07-14', '2022-10-13', '2022-11-29', '2022-12-21', '2023-01-13']
top_set_syms_n_freq: [('AU', 6), ('BGNE', 6), ('COLL', 6), ('CROX', 6), ('EXAS', 6), ('HZNP', 6), ('ACGL', 5), ('AVEO', 5), ('BABA', 5), ('BMA', 5), ('BVH', 5), ('SHYF', 5), ('TPX', 5), ('ACIW', 4), ('AGYS', 4), ('AJRD', 4), ('ANF', 4), ('CLF', 4), ('EDU', 4), ('GE', 4), ('HEES', 4), ('HSBC', 4), ('WYNN', 4), ('AAWW', 3), ('ACLS', 3), ('ATCO', 3), ('AXON', 3), ('BA', 3), ('BKE', 3), ('BURL', 3)]
top_set_syms: ['AU', 'BGNE', 'COLL', 'CROX', 'EXAS']


In [ ]:
# import webbrowser

# y_quote = 'https://finance.yahoo.com/quote/'
# for sym in top_set_syms:
#   url = y_quote + sym
#   print(url)
#   webbrowser.open(url)  # open symbols in yahoo finance

In [ ]:
# # Importing Pandas to create DataFrame
# import pandas as pd

# my_cols = ['date', 'days_lookbacks', 'n_top_syms', 'syms_start', 'syms_end', 'grp_dates', 'top_set_syms_n_freq', 'top_set_syms']
# # Creating Empty DataFrame and Storing it in variable df
# df = pd.DataFrame(columns=my_cols)

In [64]:
from myUtils import pickle_load, pickle_dump

path_dir = "C:/Users/ping/MyDrive/stocks/yfinance/"
path_data_dump = path_dir + "VSCode_dump/"
fp_top_set_syms = 'df_top_set_syms'  # df to store top set symbols

In [65]:
df = pickle_load(path_data_dump, fp_top_set_syms)
df

,date,days_lookbacks,n_top_syms,syms_start,syms_end,grp_dates,top_set_syms_n_freq,top_set_syms


In [66]:
row_add = [grp_dates[-1], days_lookbacks, n_top_syms, syms_start, syms_end, grp_dates, top_set_syms_n_freq, top_set_syms]
df.loc[len(df)] = row_add
df

,date,days_lookbacks,n_top_syms,syms_start,syms_end,grp_dates,top_set_syms_n_freq,top_set_syms
0,2023-01-13,"[16, 32, 64, 128]",30,0,5,"[2022-07-14, 2022-10-13, 2022-11-29, 2022-12-2...","[(AU, 6), (BGNE, 6), (COLL, 6), (CROX, 6), (EX...","[AU, BGNE, COLL, CROX, EXAS]"


In [67]:
pickle_dump(df, path_data_dump, fp_top_set_syms)

In [68]:
_df = pickle_load(path_data_dump, fp_top_set_syms)
_df

,date,days_lookbacks,n_top_syms,syms_start,syms_end,grp_dates,top_set_syms_n_freq,top_set_syms
0,2023-01-13,"[16, 32, 64, 128]",30,0,5,"[2022-07-14, 2022-10-13, 2022-11-29, 2022-12-2...","[(AU, 6), (BGNE, 6), (COLL, 6), (CROX, 6), (EX...","[AU, BGNE, COLL, CROX, EXAS]"
